In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
talents = pd.read_csv('game_data/Character.Talents.Info.csv')

In [4]:
os.makedirs('processed_game_data', exist_ok=True)
characters_info = pd.read_csv('game_data/Character.Info2.csv').filter(['ID', 'RaceID', 'Speed']).drop_duplicates()
tmp = pd.read_csv('game_data/Character.Levels.Stats.csv')
characters_info = pd.merge(characters_info, tmp, on='ID')
tmp = pd.read_csv('game_data/Character.Levels.Info.csv').filter(['Level', 'RequiredTokens'])
characters_info = pd.merge(characters_info, tmp, on='Level')
characters_info['nft'] = characters_info['ID'].str.startswith('NFT_')
tmp = pd.read_csv('game_data/PricePool.csv')
tmp1 = tmp[(tmp['ID'].str.startswith('price_ch_upgrade_lvl_')) & (tmp['ItemID']=='Shards')]
tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
tmp1['Level'] = tmp1['Level'].astype(int)
tmp1['nft'] = False
tmp1 = tmp1.rename(columns={'Amount': 'Shards'})
tmp1['Crown'] = 0
tmp1 = tmp1.filter(['Level', 'Shards', 'Crown', 'nft'])
tmp2 = tmp[tmp['ID'].str.startswith('price_ch_nft_upgrade_lvl_')]
tmp2['Level'] = tmp2['ID'].str.extract('(\d+)')
tmp2['Level'] = tmp2['Level'].astype(int)
tmp2 = tmp2.filter(['Level', 'ItemID', 'Amount'])
tmp2 = tmp2.pivot(index='Level', columns='ItemID', values='Amount').reset_index()
tmp2['nft'] = True
tmp = pd.concat([tmp1, tmp2])
characters_info = pd.merge(characters_info, tmp, on=['Level', 'nft'])
characters_upgrade = characters_info.filter(['Level', 'RequiredTokens', 'Shards', 'Crown', 'nft']).rename(columns={'RequiredTokens': 'Tokens'}).drop_duplicates()
characters_info = characters_info.drop(columns=['RequiredTokens', 'Shards', 'Crown'])
characters_upgrade.to_csv('processed_game_data/characters_upgrade.csv', index=False)
characters_info = characters_info[~characters_info['ID'].isin(['NFT_Shark_Bronze', 'NFT_Shark_Silver'])]
def foo(row):
    names = {
        'Ape': 'Rocky',
        'Bear': 'Bonnie',
        'Cat': 'Cranky',
        'Tiger': 'Alex',
        'Croco': 'Aldo',
        'Rabbit': 'Lola',
        'Snail': 'Zoom',
        'Shark': 'Finn',
        'Squirrel': 'Zayn',
        'Bull': 'Brutus',
        'Chicken': 'Chick'
    }
    if row['ID'].startswith('FREE_'):
        return names[row['RaceID']]
    if row['ID'].startswith('NFT_Ape'):
        return row['ID'].split('_')[-1] + ' Genesis Ape'
    if row['ID'].startswith('NFT_Shark'):
        return row['ID'].split('_')[-1] + ' Space Shark'
    return np.nan
characters_info['ID'] = characters_info.apply(foo, axis=1)
characters_info = characters_info.rename(columns={'ID': 'Name', 'RaceID': 'Race'})
characters_info.to_csv('processed_game_data/characters_info.csv', index=False)

/tmp/ipykernel_519290/4108046153.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
/tmp/ipykernel_519290/4108046153.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['Level'] = tmp1['Level'].astype(int)
/tmp/ipykernel_519290/4108046153.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [110]:
weaponData = pd.read_csv('game_data/Weapon.Data.csv')
weaponInfo = pd.read_csv('game_data/Weapon.Info.csv')
weaponInfo_dict = {}
for weapon_type in weaponInfo['WeaponType'].unique():
    weaponInfo_dict[weapon_type] = {}
    for _, row in weaponInfo[weaponInfo['WeaponType']==weapon_type].iterrows():
        weaponInfo_dict[weapon_type][row['StatType']] = row['StatValue']
weaponInfo = pd.DataFrame.from_dict(weaponInfo_dict, orient='index').reset_index().rename(columns={'index': 'WeaponType'})
weaponInfo = weaponInfo.drop(columns=weaponInfo.columns[weaponInfo.isna().any()].tolist())
weaponData = weaponData.filter(['ID', 'WeaponType', 'CollectionID', 'StarLevel', 'IsNFT', 'DamageIncrease'])
weaponInfo = pd.merge(weaponInfo, weaponData, on='WeaponType').drop(columns=['WeaponType'])
weaponInfo['level'] = 1
for lvl in range(2, 21):
    tmp = weaponInfo[weaponInfo['level']==1]
    tmp['level'] = lvl
    tmp['weapon_damage'] = tmp.apply(lambda row: float(row['weapon_damage']) + float(row['DamageIncrease'])*lvl, axis=1)
    weaponInfo = pd.concat([weaponInfo, tmp])
weaponInfo = weaponInfo.drop(columns=['DamageIncrease'])
weaponInfo.to_csv('processed_game_data/weapon_info.csv', index=False)
weaponInfo = pd.read_csv('processed_game_data/weapon_info.csv')
tmp = pd.read_csv('game_data/Weapon.Data.csv').filter(['ID', 'StatType', 'StatValue'])
tmp = tmp[tmp['StatType'].isin(weaponInfo.columns)]
tmp['StarLevel'] = tmp['ID'].str.extract('(\d+)')
tmp['StarLevel'] = tmp['StarLevel'].astype(int)
tmp = tmp.sort_values('StarLevel')
for i, row in tmp.iterrows():
    statType = row['StatType']
    statValue = row['StatValue']
    starLevel = row['StarLevel']
    id = '_'.join(row['ID'].split('_')[:-1])
    if statType == 'weapon_damage':
        weaponInfo['weapon_damage'] = weaponInfo.apply(lambda x: x['weapon_damage'] + statValue if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_damage'], axis=1)
    if statType == 'weapon_projectile_range':
        weaponInfo['weapon_projectile_range'] = weaponInfo.apply(lambda x: x['weapon_projectile_range'] + statValue if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_projectile_range'], axis=1)
weaponInfo.to_csv('processed_game_data/weapon_info.csv', index=False)

/tmp/ipykernel_519290/1398389837.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['level'] = lvl
/tmp/ipykernel_519290/1398389837.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['weapon_damage'] = tmp.apply(lambda row: float(row['weapon_damage']) + float(row['DamageIncrease'])*lvl, axis=1)


In [111]:
weaponInfo

,weapon_class,weapon_damage,weapon_cooldownAfterLastBullet,weapon_reloadTime,weapon_ammo,weapon_projectile_speed,weapon_projectile_range,weapon_projectile_spawn_delay_after_shot,ID,CollectionID,StarLevel,IsNFT,level
0,Spread,2200.0,0.65,2.1,3,14.0,9.0,0.1,WeaponType_Shotgun_Default_Star1,Default,1,False,1
1,Spread,2300.0,0.65,2.1,3,14.0,9.0,0.1,WeaponType_Shotgun_Default_Star2,Default,2,False,1
2,Spread,2300.0,0.65,2.1,3,14.0,9.0,0.1,WeaponType_Shotgun_Default_Star3,Default,3,False,1
3,Spread,2300.0,0.65,2.1,3,14.0,9.0,0.1,WeaponType_Shotgun_Default_Star4,Default,4,False,1
4,Spread,2300.0,0.65,2.1,3,14.0,9.0,0.1,WeaponType_Shotgun_Default_Star5,Default,5,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Precision,1600.0,0.75,2.4,2,12.0,10.0,0.1,WeaponType_Crossbow_Mythical_Star2,Mythical,2,True,20
1916,Precision,1800.0,0.75,2.4,2,12.0,10.0,0.1,WeaponType_Crossbow_Mythical_Star3,Mythical,3,True,20
1917,Precision,2000.0,0.75,2.4,2,12.0,10.0,0.1,WeaponType_Crossbow_Mythical_Star4,Mythical,4,True,20
1918,Precision,2300.0,0.75,2.4,2,12.0,10.0,0.1,WeaponType_Crossbow_Mythical_Star5,Mythical,5,True,20
